In [1]:
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
twitter_df = pd.read_csv(r'E:\Python\Datasets\Disaster_Tweets\train.csv')
twitter_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [78]:
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     7613 non-null   int64 
 1   keyword                7552 non-null   object
 2   location               5080 non-null   object
 3   text                   7613 non-null   object
 4   target                 7613 non-null   int64 
 5   cleaned_text           7613 non-null   object
 6   tokenized              7613 non-null   object
 7   stopwords_removed      7613 non-null   object
 8   porter_stemmer         7613 non-null   object
 9   lemmatize_word         7613 non-null   object
 10  porter_stemmer_joined  7613 non-null   object
 11  lemmatize_word_joined  7613 non-null   object
dtypes: int64(2), object(10)
memory usage: 713.8+ KB


In [4]:
twitter_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

## Text preprocessing

#### 1. Lower case

In [5]:
# Lower case

twitter_df['cleaned_text'] = twitter_df['text'].apply(lambda x: x.lower())
twitter_df.head()

,id,keyword,location,text,target,cleaned_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this #earthquake m...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask. canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby #alaska as ...


#### 2. Expand the Contrations

In [6]:
# Expand the Contrations
import contractions

In [7]:
# Test
contractions.fix("I'd like to know how I'd done that!")

'I would like to know how I would done that!'

In [8]:
# Expanding the Contrations
twitter_df['cleaned_text'] = twitter_df['cleaned_text'].apply(contractions.fix)

In [9]:
# Checking contractions
print(twitter_df["text"][67])
print(twitter_df["cleaned_text"][67])

'I can't have kids cuz I got in a bicycle accident &amp; split my testicles. it's impossible for me to have kids' MICHAEL YOU ARE THE FATHER
'i can not have kids cuz i got in a bicycle accident &amp; split my testicles. it is impossible for me to have kids' michael you are the father


#### 3. Noise Removal

3.1 Remove URLs

In [10]:
import re
def remove_URL(text):
    """
        Remove URLs from a simple string
    """
    return re.sub(r"https?://\S+|www\.\S+", "", text)

In [11]:
# remove url from the text
twitter_df['cleaned_text'] = twitter_df['cleaned_text'].apply(remove_URL)

In [12]:
# Checking
print(twitter_df["text"][31])
print(twitter_df["cleaned_text"][31])

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
@bbcmtd wholesale markets ablaze 


3.2 Remove HTML tags

In [13]:
def remove_html_tag(text):
    """
        Remove html tags from a simple text
    """
    html_tag = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html_tag, "", text)

In [14]:
# remove html tags from text
twitter_df['cleaned_text'] = twitter_df['cleaned_text'].apply(remove_html_tag)

In [15]:
# Checking
print(twitter_df["text"][62])
print(twitter_df["cleaned_text"][62])

Rene Ablaze &amp; Jacinta - Secret 2k13 (Fallen Skies Edit) - Mar 30 2013  https://t.co/7MLMsUzV1Z
rene ablaze  jacinta - secret 2k13 (fallen skies edit) - mar 30 2013  


3.3 Remove Non-ASCI:

In [16]:
def removal_non_ascii(text):
    return re.sub(r'[^\x00-\x7f]', '', text)

In [17]:
twitter_df['cleaned_text'] = twitter_df['cleaned_text'].apply(removal_non_ascii)

In [18]:
# Checking
print(twitter_df["text"][38])
print(twitter_df["cleaned_text"][38])

Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J
barbados #bridgetown jamaica  two cars set ablaze: santa cruz  head of the st elizabeth police superintende...  


#### 4. Remove punctuations

In [19]:
import string
def remove_punc(text):
#     return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)
    return text.translate(str.maketrans('', '', string.punctuation))

In [20]:
twitter_df['cleaned_text'] = twitter_df['cleaned_text'].apply(remove_punc)

In [21]:
# Checking
print(twitter_df["text"][5])
print(twitter_df["cleaned_text"][5])

#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
rockyfire update  california hwy 20 closed in both directions due to lake county fire  cafire wildfires


#### 5. Tokeinzation

In [22]:
from nltk.tokenize import word_tokenize

twitter_df['tokenized'] = twitter_df['cleaned_text'].apply(word_tokenize)
twitter_df.head()

,id,keyword,location,text,target,cleaned_text,tokenized
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[our, deeds, are, the, reason, of, this, earth..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfires, evacuation..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[just, got, sent, this, photo, from, ruby, ala..."


#### 6.Removing stopwords

In [23]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maksb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
stop = set(stopwords.words('english'))
twitter_df['stopwords_removed'] = twitter_df['tokenized'].apply(lambda x: [word for word in x if word not in stop])
twitter_df.head()

,id,keyword,location,text,target,cleaned_text,tokenized,stopwords_removed
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi..."


#### 7. Stemming

In [25]:
from nltk.stem import PorterStemmer

def porter_stemmer(text):
    
    stemmer = PorterStemmer()
    stems = [stemmer.stem(word) for word in text]
    return stems

In [26]:
twitter_df['porter_stemmer'] = twitter_df['stopwords_removed'].apply(porter_stemmer)
twitter_df.head()

,id,keyword,location,text,target,cleaned_text,tokenized,stopwords_removed,porter_stemmer
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...","[deed, reason, earthquak, may, allah, forgiv, us]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, rong, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...","[resid, ask, shelter, place, notifi, offic, ev..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...","[13000, peopl, receiv, wildfir, evacu, order, ..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, sent, photo, rubi, alaska, smoke, wildfi..."


### 8. Lemmatization

In [27]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

In [28]:
lemmatizer = WordNetLemmatizer()

twitter_df['lemmatize_word'] = twitter_df['stopwords_removed'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
twitter_df.head()

,id,keyword,location,text,target,cleaned_text,tokenized,stopwords_removed,porter_stemmer,lemmatize_word
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,"[our, deeds, are, the, reason, of, this, earth...","[deeds, reason, earthquake, may, allah, forgiv...","[deed, reason, earthquak, may, allah, forgiv, us]","[deed, reason, earthquake, may, allah, forgive..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, ronge, sask, canada]","[forest, fire, near, la, rong, sask, canada]","[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,"[all, residents, asked, to, shelter, in, place...","[residents, asked, shelter, place, notified, o...","[resid, ask, shelter, place, notifi, offic, ev...","[resident, asked, shelter, place, notified, of..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[13000, people, receive, wildfires, evacuation...","[13000, people, receive, wildfires, evacuation...","[13000, peopl, receiv, wildfir, evacu, order, ...","[13000, people, receive, wildfire, evacuation,..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,"[just, got, sent, this, photo, from, ruby, ala...","[got, sent, photo, ruby, alaska, smoke, wildfi...","[got, sent, photo, rubi, alaska, smoke, wildfi...","[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [29]:
twitter_df['porter_stemmer_joined'] = twitter_df['porter_stemmer'].apply(lambda x: ' '.join(x))
twitter_df['lemmatize_word_joined'] = twitter_df['lemmatize_word'].apply(lambda x: ' '.join(x))

### 9. CountVectorizer

In [30]:
from sklearn.pipeline import Pipeline

In [31]:
def check_preporcessing_methods_with_prediction(X, y, alpha = list(np.linspace(1,50,10))):
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state = 1)
    pipeline = Pipeline([('CountVectorizer', feature_extraction.text.CountVectorizer()), ('classifier', linear_model.RidgeClassifier())])
    parametrs = {'classifier__alpha': alpha,
             'classifier__class_weight': [None, 'balanced'],
             'classifier__copy_X': [True],
             'classifier__fit_intercept': [True],
             'classifier__max_iter': [None],
             'classifier__normalize': [False],
             'classifier__random_state': [0],
             'classifier__solver': ['auto'],
             'classifier__tol': [0.001]}
    clf_grid = GridSearchCV(pipeline, param_grid=parametrs, cv = 3)
    clf_grid.fit(X_train, y_train)
    print('Best Score:', clf_grid.best_score_)
    print('Best Params', clf_grid.best_params_)

In [32]:
check_preporcessing_methods_with_prediction(twitter_df['porter_stemmer_joined'],twitter_df['target'], list(np.linspace(7,16,15)) )

Best Score: 0.7961114030478192
Best Params {'classifier__alpha': 14.071428571428573, 'classifier__class_weight': None, 'classifier__copy_X': True, 'classifier__fit_intercept': True, 'classifier__max_iter': None, 'classifier__normalize': False, 'classifier__random_state': 0, 'classifier__solver': 'auto', 'classifier__tol': 0.001}


In [33]:
check_preporcessing_methods_with_prediction(twitter_df['lemmatize_word_joined'],twitter_df['target'] ,list(np.linspace(24,28,15)))

Best Score: 0.7999649675950254
Best Params {'classifier__alpha': 27.428571428571427, 'classifier__class_weight': 'balanced', 'classifier__copy_X': True, 'classifier__fit_intercept': True, 'classifier__max_iter': None, 'classifier__normalize': False, 'classifier__random_state': 0, 'classifier__solver': 'auto', 'classifier__tol': 0.001}


In [34]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(twitter_df['text'], twitter_df['target'], random_state = 0)

In [35]:
count_vectorizer = feature_extraction.text.CountVectorizer()
train_vectors = count_vectorizer.fit_transform(X_train)
test_vectors = count_vectorizer.transform(X_test)

In [36]:
ridge_model_clf = linear_model.RidgeClassifier()

In [37]:
cv_scores = model_selection.cross_val_score(ridge_model_clf, train_vectors, y_train, cv = 3, scoring='f1')

In [38]:
cv_scores

array([0.70415335, 0.72426938, 0.7343245 ])

In [39]:
ridge_model_clf.fit(train_vectors, y_train)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [40]:
prediction = ridge_model_clf.predict(test_vectors)

In [41]:
from sklearn.metrics import f1_score

In [42]:
f1_score(prediction, y_test)

0.7390728476821192

In [43]:
twitter_test = pd.read_csv(r'E:\Python\Datasets\Disaster_Tweets\test.csv')


In [44]:
twitter_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [45]:
ridge_model_clf.get_params()

{'alpha': 1.0,
 'class_weight': None,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [46]:


pipeline = Pipeline([('CountVectorizer', feature_extraction.text.CountVectorizer()), ('classifier', linear_model.RidgeClassifier())])

In [47]:
parametrs = {'classifier__alpha': list(np.linspace(1,50,10)),
             'classifier__class_weight': [None, 'balanced'],
             'classifier__copy_X': [True],
             'classifier__fit_intercept': [True],
             'classifier__max_iter': [None],
             'classifier__normalize': [False],
             'classifier__random_state': [0],
             'classifier__solver': ['auto'],
             'classifier__tol': [0.001]}
clf_grid = GridSearchCV(pipeline, param_grid=parametrs, cv = 3)
clf_grid.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('CountVectorizer',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                 

In [48]:
clf_grid.best_score_

0.8001401296198984

In [49]:
clf_grid.best_params_

{'classifier__alpha': 17.333333333333336,
 'classifier__class_weight': None,
 'classifier__copy_X': True,
 'classifier__fit_intercept': True,
 'classifier__max_iter': None,
 'classifier__normalize': False,
 'classifier__random_state': 0,
 'classifier__solver': 'auto',
 'classifier__tol': 0.001}

In [50]:
clf_best = linear_model.RidgeClassifier(alpha = 21)

In [51]:
clf_best.fit(train_vectors, y_train)


RidgeClassifier(alpha=21, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [52]:
f1_score(clf_best.predict(test_vectors),y_test)

0.761049723756906

In [53]:
cv_scores = model_selection.cross_val_score(clf_best, train_vectors, y_train, cv = 3, scoring='f1')
cv_scores.mean()

0.7451708565758928

In [54]:
clf_grid.best_params_

{'classifier__alpha': 17.333333333333336,
 'classifier__class_weight': None,
 'classifier__copy_X': True,
 'classifier__fit_intercept': True,
 'classifier__max_iter': None,
 'classifier__normalize': False,
 'classifier__random_state': 0,
 'classifier__solver': 'auto',
 'classifier__tol': 0.001}

In [55]:
parametrs = {'alpha': [0.1],
             'class_weight': [None, 'balanced'],
             'copy_X': [True],
             'fit_intercept': [True],
             'max_iter': [None],
             'normalize': [False],
             'random_state': [0],
             'solver': ['auto'],
             'tol': [0.001]}
clf_grid = GridSearchCV(ridge_model_clf, param_grid=parametrs, cv = 3)
clf_grid.fit(train_vectors, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RidgeClassifier(alpha=1.0, class_weight=None,
                                       copy_X=True, fit_intercept=True,
                                       max_iter=None, normalize=False,
                                       random_state=None, solver='auto',
                                       tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.1], 'class_weight': [None, 'balanced'],
                         'copy_X': [True], 'fit_intercept': [True],
                         'max_iter': [None], 'normalize': [False],
                         'random_state': [0], 'solver': ['auto'],
                         'tol': [0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [56]:
clf_grid.best_score_

0.7565247854265196

### 10. TF-IDF Vectorizer

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

def TFIDF(data, ngram = 1):
    tfidf_x = TfidfVectorizer(ngram_range = (ngram, ngram))
    emb = tfidf_x.fit_transform(data).toarray()
    return emb, tfidf_x

In [82]:
def check_preporcessing_methods_with_prediction(X, y, alpha = list(np.linspace(1,50,10))):
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state = 1)
    pipeline = Pipeline([('TfidfVectorizer', TfidfVectorizer()), ('classifier', linear_model.RidgeClassifier())])
    parametrs = {'classifier__alpha': alpha,
             'classifier__class_weight': [None, 'balanced'],
             'classifier__copy_X': [True],
             'classifier__fit_intercept': [True],
             'classifier__max_iter': [None],
             'classifier__normalize': [False],
             'classifier__random_state': [0],
             'classifier__solver': ['auto'],
             'classifier__tol': [0.001],
             'TfidfVectorizer__ngram_range': [(1,1), (1,2), (1,3)]}
    clf_grid = GridSearchCV(pipeline, param_grid=parametrs, cv = 3, scoring= 'f1')
    clf_grid.fit(X_train, y_train)
    print('Best Score:', clf_grid.best_score_)
    print('Best Params', clf_grid.best_params_)

In [83]:
check_preporcessing_methods_with_prediction(twitter_df['porter_stemmer_joined'],twitter_df['target'], list(np.linspace(0.1,20,15)) )

Best Score: 0.756213792099086
Best Params {'TfidfVectorizer__ngram_range': (1, 2), 'classifier__alpha': 1.5214285714285714, 'classifier__class_weight': 'balanced', 'classifier__copy_X': True, 'classifier__fit_intercept': True, 'classifier__max_iter': None, 'classifier__normalize': False, 'classifier__random_state': 0, 'classifier__solver': 'auto', 'classifier__tol': 0.001}


In [84]:
check_preporcessing_methods_with_prediction(twitter_df['lemmatize_word_joined'],twitter_df['target'], list(np.linspace(0.1,20,15)) )

Best Score: 0.754434066383692
Best Params {'TfidfVectorizer__ngram_range': (1, 2), 'classifier__alpha': 5.785714285714285, 'classifier__class_weight': 'balanced', 'classifier__copy_X': True, 'classifier__fit_intercept': True, 'classifier__max_iter': None, 'classifier__normalize': False, 'classifier__random_state': 0, 'classifier__solver': 'auto', 'classifier__tol': 0.001}


In [85]:
set([1,2,3])[0]


TypeError: 'set' object is not subscriptable

## Making submission prediction

In [63]:
count_vectorizer_sumbission = feature_extraction.text.CountVectorizer()

submission_train_vectors = count_vectorizer_sumbission.fit_transform(twitter_df['lemmatize_word_joined'])


In [66]:
TfidfVectorizer_sumbission = TfidfVectorizer(ngram_range = (1, 2))
submission_train_vectors = TfidfVectorizer_sumbission.fit_transform(twitter_df['lemmatize_word_joined'])


In [67]:
# Preprocessing
twitter_test = pd.read_csv(r'E:\Python\Datasets\Disaster_Tweets\test.csv')

twitter_test['cleaned_text'] = twitter_test['text'].apply(lambda x: x.lower())
twitter_test['cleaned_text'] = twitter_test['cleaned_text'].apply(contractions.fix)
twitter_test['cleaned_text'] = twitter_test['cleaned_text'].apply(remove_URL)
twitter_test['cleaned_text'] = twitter_test['cleaned_text'].apply(remove_html_tag)
twitter_test['cleaned_text'] = twitter_test['cleaned_text'].apply(removal_non_ascii)
twitter_test['cleaned_text'] = twitter_test['cleaned_text'].apply(remove_punc)
twitter_test['tokenized'] = twitter_test['cleaned_text'].apply(word_tokenize)
twitter_test['stopwords_removed'] = twitter_test['tokenized'].apply(lambda x: [word for word in x if word not in stop])
twitter_test['lemmatize_word'] = twitter_test['stopwords_removed'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
twitter_test['lemmatize_word_joined'] = twitter_test['lemmatize_word'].apply(lambda x: ' '.join(x))


In [65]:
submission_test_vectors = count_vectorizer_sumbission.transform(twitter_test['lemmatize_word_joined'])

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [68]:
submission_test_vectors = TfidfVectorizer_sumbission.transform(twitter_test['lemmatize_word_joined'])

In [69]:
clf_best = linear_model.RidgeClassifier(alpha = 1.5)
clf_best.fit(submission_train_vectors, twitter_df['target'])
submission_prediction = clf_best.predict(submission_test_vectors)

In [70]:
submission_prediction

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [71]:
twitter_test.head()

,id,keyword,location,text,cleaned_text,tokenized,stopwords_removed,lemmatize_word,lemmatize_word_joined
0,0,NaN,NaN,Just happened a terrible car crash,just happened a terrible car crash,"[just, happened, a, terrible, car, crash]","[happened, terrible, car, crash]","[happened, terrible, car, crash]",happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...,"[heard, about, earthquake, is, different, citi...","[heard, earthquake, different, cities, stay, s...","[heard, earthquake, different, city, stay, saf...",heard earthquake different city stay safe ever...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...,"[there, is, a, forest, fire, at, spot, pond, g...","[forest, fire, spot, pond, geese, fleeing, acr...","[forest, fire, spot, pond, goose, fleeing, acr...",forest fire spot pond goose fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires,"[apocalypse, lighting, spokane, wildfires]","[apocalypse, lighting, spokane, wildfires]","[apocalypse, lighting, spokane, wildfire]",apocalypse lighting spokane wildfire
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills 28 in china and taiwan,"[typhoon, soudelor, kills, 28, in, china, and,...","[typhoon, soudelor, kills, 28, china, taiwan]","[typhoon, soudelor, kill, 28, china, taiwan]",typhoon soudelor kill 28 china taiwan


In [72]:
submission_df = twitter_test[['id']]

In [73]:
submission_df.head()

,id
0,0
1,2
2,3
3,9
4,11


In [74]:
submission_df = twitter_test[['id']].copy()
submission_df['target'] = submission_prediction

In [75]:
submission_df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [76]:
submission_df.to_csv(r'E:\Python\Datasets\Disaster_Tweets\submission.csv', index=False)